In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
m = nn.Conv2d(2, 1, (2, 2), stride=1, padding=0)

In [3]:
k1 = torch.tensor(
    [[1, -2],
     [2, 0]]
).float()
k2 = torch.tensor(
    [[-3, 4],
    [5, 3]]
).float()
kernel = torch.stack((k1, k2), dim=0).unsqueeze(0)

In [4]:
m.weight = nn.Parameter(kernel)

In [5]:
m.bias = nn.Parameter(torch.tensor([0]).float())

In [6]:
i1 = torch.tensor(
    [[1, 2, 3],
     [0, 2, -1],
     [2, 3, 1]]
).float()
i2 = torch.tensor(
    [[1, 3, -2],
     [2, 4, 0],
     [3, 3, 1]]
).float()
image = torch.stack((i1, i2))

In [7]:
image, m(image)

(tensor([[[ 1.,  2.,  3.],
          [ 0.,  2., -1.],
          [ 2.,  3.,  1.]],
 
         [[ 1.,  3., -2.],
          [ 2.,  4.,  0.],
          [ 3.,  3.,  1.]]]),
 tensor([[[28.,  3.],
          [34., 16.]]], grad_fn=<SqueezeBackward1>))

In [8]:
class Conv2dSimple:
    
    def __init__(self, in_channel, out_channel, kernel_size, stride=1, padding=0):
        self.stride = stride
        self.padding = padding
        self.out_channel = out_channel
        self.kernel_size = kernel_size
        self.weight = torch.randn((out_channel, in_channel) + kernel_size)
        self.bias = torch.randn(out_channel)
        
    def __call__(self, x):
        data = x.unsqueeze(0) if len(x.shape) == 3 else x
        # B表示批量大小，I表示in_channel个数，H表示高度，W表示宽度
        B, I, H, W = data.shape
        h_step = (H + 2 * self.padding - self.kernel_size[0]) // self.stride + 1
        w_step = (W + 2 * self.padding - self.kernel_size[1]) // self.stride + 1
        # 定义输出的形状
        output = torch.zeros(B, self.out_channel, h_step, w_step)
        # 在图像的边缘增加0
        data = F.pad(data, (self.padding,) * 4)
        for i in range(h_step):
            h_begin = i * self.stride
            h_end = h_begin + self.kernel_size[0]
            for j in range(w_step):
                w_being = j * self.stride
                w_end = w_being + self.kernel_size[1]
                inputs = data[:, :, h_begin: h_end, w_being: w_end].unsqueeze(1)
                # inputs的形状：     （B,           1, I, kernel_size[0], kernel_size[1]）
                # self.weight的形状：（   out_channel, I, kernel_size[0], kernel_size[1]）
                # linear_out的形状：  (B, out_channel）
                # bias的形状：        (   out_channel）
                linear_out = (inputs * self.weight).sum((-3, -2, -1))
                output[:, :, i, j] = linear_out + self.bias
        self.out = output.squeeze(0) if len(x.shape) == 3 else output
        return self.out
    
    def parameters(self):
        return [self.weight, self.bias]

In [9]:
t = Conv2dSimple(2, 1, (2, 2), stride=1, padding=0)
t.weight = nn.Parameter(kernel)
t.bias = nn.Parameter(torch.tensor([0]).float())

In [10]:
image, t(image)

(tensor([[[ 1.,  2.,  3.],
          [ 0.,  2., -1.],
          [ 2.,  3.,  1.]],
 
         [[ 1.,  3., -2.],
          [ 2.,  4.,  0.],
          [ 3.,  3.,  1.]]]),
 tensor([[[28.,  3.],
          [34., 16.]]], grad_fn=<SqueezeBackward1>))